# SVD to Improve Recommendation Engines

TODO:
1. Write introduction
2. Some bug in cosine similarity measure. 

__Source__: Simplifying Data with Singular Value Decomposition (Chapter 14) in Machine Learning in Action

__Out of Scope__: Implementation of SVD. We'll use `linalg` in Numpy for factorization.

### Introduction

- TODO: Write an introduction in 2-3 lines


Singular Value Decomposition (SVD) is a __matrix factorization__ algorithm which helps us distill distill hidden information from data. It finds application in number of fields ranging from bioinformatics to finance.  

The idea of SVD is to represent the original data set with a compressed data set by removing noise and other redundant information. The latter is expected to hold just enough information which is needed for our algorithm to yield good results. Not only does the algorithm improve in terms of accuracy but improvement is also in terms of run time. However, as a downside, the transformed data is not so easy to comprehend. 

### Brief history of SVD
SVD was first used in the field of information retrieval. _Latent Semantic Analysis (LSA)_ or _Latent Semantic Indexing (LSI)_  was the first algorithm to have used SVD. LSI is a method to search for words in a document. We construct a document-word matrix. However, a simple search of a word in a document misses the point that words may be mis-spelt or a synonym of the same word might have been used for the same concept. This is where SVD comes into play. SVD creates a set of _singular values_ which represents concepts or topics contained in the document. Now, instead of doing a vanilla search of words in the documents, we'll be searching in the concept space.  

### SVD in Recommendation Systems
Simple versions of recommendation systems compute similarity between users or items. However, advanced versions of the same use SVD to create a concept space from the data and then compute similarities between users/ items in this concept space. 

As an example, consider a matrix containing the ratings of 7 users for 5 restaurants. After applying SVD, let's assume we get a 2D concept space. This space broadly represents the dimensions along which user rating varied.  

### Matrix Factorization
As noted earlier, quite often only a few data points in a given dataset contains the relevant information. The other data points can be construed as noise or irrelevant. The data points of interest can be obtained by factorizing the original matrix. 

SVD takes an original matrix $Data$ of size $m*n$ and decomposes it into three matrices viz. $U, \sum, V$ of dimensions $m*m$, $m*n$ and $n*n$ respectively. 

$$Data_{m*n} = U_{m*m} \sum_{m*n} V_{n*n}^{T}$$

SVD creates a $\sum$ matrix which is a diagonal matrix. By convention, the diagonal elements in $\sum$ are sorted in descending order. These diagonal elements are called singular values and they correspond to the singular values of our original dataset $Data$. 

#### Relation between singular values and eigen values
PCA gives us the eigenvalues of a matrix which tells us the most important features in our dataset. The same thing is true about the singular values in $\sum$. Singular values are the square root of the eigenvalues of $Data*Data^{T}$.

#### Insignificant singular values
Not all the singular values we'll obtain will be significant. Many of these will either be zero or negligibly small. We will see its relevance later in the code. 

#### Practical points for SVD
1. `linalg.svd` in Numpmy returns $\sum$ as a row vector as it is a diagonal matrix. It is done with a view to saving space. 
2. _Heuristics for keeping optimal number of singular values_: Similar to PCA, keep 90% of the energy expressed by the $\sum$ matrix. To calculate the total energy, add up all the squared singular values. Then add squared singular value until the sum reaches 90% of the total.  

### Collaborative filtering based recommendation engines

1. _Collaborative filtering_ based recommendation engines work by taking a data set of users' data and comparing it with the data of other users. Instead of trying to describe the similarity between items based on some attributes that an expert considers as important, we compare the similarity based on what people think of the items. In other words, collaborative filtering based approach doesn't care about the attributes of the items. It compares similarity strictly by the opinions of many users.  
2. We can either compare the similarity between users or similarity between items. Later in this tutorial, we'll see why item based similarity makes more sense. 

We'll be seeing the folllwing things in this section:
1. How to measure the similarity between items?
2. Trade-offs between item based and user based similarity
3. How to measure the success of a recommendation engine?

#### Similarity metrics
1. __Based on Euclidean distance__: $similarity = 1/ (1+distance)$. If $distance=0$, $similarity = 1$. If $distance$ = large number, $similarity=0$.
2. __Pearson correlation__: It is insensitive to the magnitude of the rating given by users. Value ranges from -1 to 1. It can be normalized so that it falls between 0 and 1 by using $0.5 + 0.5*corrcoef()$. 
3. __Cosine similarity__: Measures the cosine of the angle between two vectors. Ranges between -1 and 1. To be normalized in the same manner as explained in the previous point. 

In [1]:
def loadExData():
    return[[0, 0, 0, 2, 2],
           [0, 0, 0, 3, 3],
           [0, 0, 0, 1, 1],
           [1, 1, 1, 0, 0],
           [2, 2, 2, 0, 0],
           [5, 5, 5, 0, 0],
           [1, 1, 1, 0, 0]]

def loadExData2():
    return[[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
           [0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
           [0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
           [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
           [5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
           [0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
           [4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
           [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
           [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
           [0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
           [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]]

__Note:__ In the similarity functions below, we will pass the input vectors as a column vector. This is with a view to using item based recommendation. The dataset to be used has users along the row and items along the column. 

In [17]:
import numpy as np
from numpy import linalg as la

def euclidean_similarity(inpA, inpB):
    return 1.0/(1.0 + la.norm(inpA-inpB))

def pearson_similarity(inpA, inpB):
    if len(inpA) < 3: 
        return 1.0
    else:
        return 0.5 + 0.5*np.corrcoef(inpA, inpB, rowvar = 0)[0][1]
    
def cosine_similarity(inpA, inpB):
    numerator = np.inner(inpA, inpB)  # float(inpA.T*inpB)
    denominator = la.norm(inpA)*la.norm(inpB)
    return 0.5 + 0.5*(numerator/denominator)

In [18]:
# Test similarity measures
myMat = np.array(loadExData())
print type(myMat)
print "Test euclidean similarity", euclidean_similarity(myMat[:,0], myMat[:,0])
print "Test pearson similarity", pearson_similarity(myMat[:,0], myMat[:,0])
print "Test cosine similarity", cosine_similarity(myMat[:,0], myMat[:,0])

<type 'numpy.ndarray'>
Test euclidean similarity 1.0
Test pearson similarity 1.0
Test cosine similarity 1.0


#### Evaluating recommendation engines 

- We use k-fold cross validation for this purpose. Note that in this case we don't know the target value to predict. Neither can we ask the user if our prediction is right. We will take some known rating and hold it out of the data. We'll then make a prediction for that value and compare it against the true value. 
- We will use RMSE to evaluate the recommendation engine. If the ratings are on a scale of 1 through 5, an RMSE of 1 would imply that our predictions are on an average one star off what the people actually think. 

### Example: Restaurant dish recommendation engine

#### Problem statement
To recommend restaurant and food item to users. 

#### Approach
We'll first create a basic recommndation engine which looks for items a user hasn't tried. The next step is to improve the engine by using SVD to reduce the feature space.

Given a user, it will return the top N best recommendations for that user. To do this we will do the following:
1. Search for items the user in question hasn't yet rated i.e. look for 0's in the user-item matrix.
2. For all the items this user hasn't yet rated, find a projected rating for each item i.e. predicted score for that item. This is where the similarity calculation comes into play. 
3. Sort the list in descending order and return the first N items. 

In [59]:
def standEst(dataMat, user, simMeas, item):
    """
    Description: Calculates the estimated rating a user would give to an item for a 
    given similarity measure.
    input:
    dataMat: user-item rating matrix. Users along the rows and items along the column. 
    user: User for whom items are to be predicted. 
    simMeas: Similarity metric to be used. 
    item: 
    """
    n = shape(dataMat)[1]  # Get number of items in the dataset
    simTotal = 0.0; ratSimTotal = 0.0  # Variables to calculate estimated rating
    for j in range(n):  # Loop over every item
        userRating = dataMat[user,j]
        # Skip the item which the user hasn't rated. Compare the items which are rated 
        # with other items which are rated. 
        if userRating == 0: 
            continue  
        # The variable overlap captures the elements that have been rated between two
        # items. If there's no overlap, the similarity is 0 and exit the loop. Otherwise,
        # calculate the similarity based on overlapping items. 
        # overLap = nonzero(logical_and(dataMat[:,item]>0, dataMat[:,j]>0))[0]
        overLap = nonzero(logical_and(dataMat[:,item].A>0, dataMat[:,j].A>0))[0]
        if len(overLap) == 0: 
            similarity = 0
        else: 
            similarity = simMeas(dataMat[overLap,item], dataMat[overLap,j])
        print 'the %d and %d similarity is: %f' % (item, j, similarity)
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: 
        return 0
    else:
        # Normalize the similarity ratings product by dividing it by the sum of all the 
        # ratings. 
        return ratSimTotal/simTotal

In [60]:
def recommend(dataMat, user, N=3, simMeas=cosine_similarity, estMethod=standEst):
    """
    Generates the top N recommendations (N=3 by default). 
    """
    # Create a list of unrated items for a user
    unratedItems = nonzero(dataMat[user,:].A==0)[1]
    if len(unratedItems) == 0: 
        return 'User has rated all the items'
    itemScores = []
    # If there are unrated items, loop over each of those. For each unrated item, call
    # the estimation method to get the forcasted score for that item. The item's index
    # and the estimated score are placed in a list of tuples called itemScores. 
    # Finally, this list is sorted in descending order by the estimated score and 
    # returned. 
    for item in unratedItems:
        estimatedScore = estMethod(dataMat, user, simMeas, item)
        itemScores.append((item, estimatedScore))
    return sorted(itemScores, key=lambda jj: jj[1], reverse=True)[:N]

In [61]:
# Test the functions in the last two blocks
# Modify the matrix which was created earlier
myMat = mat(loadExData())
myMat[0,1] = myMat[0,0]= myMat[1,0]= myMat[2,0]= 4
myMat[3,3] = 2
print myMat

[[4 4 0 2 2]
 [4 0 0 3 3]
 [4 0 0 1 1]
 [1 1 1 2 0]
 [2 2 2 0 0]
 [5 5 5 0 0]
 [1 1 1 0 0]]


In [62]:
print type(myMat)
print "Recommendation for user 2", recommend(myMat,2)

<class 'numpy.matrixlib.defmatrix.matrix'>
Recommendation for user 2

TypeError: float argument required, not matrix

#### Improving recommendations with SVD

In [64]:
def svdEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]
    simTotal = 0.0; ratSimTotal = 0.0
    U,Sigma,VT = la.svd(dataMat)
    Sig4 = mat(eye(4)*Sigma[:4]) #arrange Sig4 into a diagonal matrix
    xformedItems = dataMat.T * U[:,:4] * Sig4.I  #create transformed items
    for j in range(n):
        userRating = dataMat[user,j]
        if userRating == 0 or j==item: continue
        similarity = simMeas(xformedItems[item,:].T,\
                             xformedItems[j,:].T)
        print 'the %d and %d similarity is: %f' % (item, j, similarity)
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal

In [66]:
recommend(myMat, 1, estMethod=svdEst)

TypeError: float argument required, not matrix

In [67]:
recommend(myMat, 1, simMeas= pearson_similarity, estMethod=svdEst)

the 1 and 0 similarity is: 0.626075
the 1 and 3 similarity is: 0.672793
the 1 and 4 similarity is: 0.614375
the 2 and 0 similarity is: 0.429334
the 2 and 3 similarity is: 0.387057
the 2 and 4 similarity is: 0.043539


[(2, 3.4992661245386785), (1, 3.3272324280613659)]

### Challenges with building recommendation engines
1. Doing SVD every time we want a projected score will make the system slow, particularly on the larger datasets. The SVD can be done once when the program is launched. On large systems, SVD is done once a day or is done offline. 
2. The user-ratings matrix will be sparse. We can save some memory and computation by storing only the non-zero values. 
3. In our program, we calculate the similarity score for multiple items each time we wanted a recommendation score. The scores are between items so we can reuse them if another user needs them. Another thing commonly done in practice is to compute the similarity score offline and store them. 
4. _Cold start problem_ refers to the problem of making recommendations when we have no data. We can make use of the attributes of the items in such cases.  In our example, we can tag the dishes with a number of parameters such as vegetarian, American BBQ, expensive and so on. We can treat these data for our similarity calculations. This is called content based recommendation. Such type is not as good as collaborative filtering based method, but it might be needed to take care of the cold start problem. 